<a href="https://colab.research.google.com/github/brendaormachea/brendaormachea.github.io/blob/main/PrimeraParte_Brenda_Ormachea.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CREACIÓN DEL NOTEBOOK Y CONEXIÓN CON EL FICHERO CSV.

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
!ls

drive  sample_data


In [28]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [29]:
!wget -q https://dlcdn.apache.org/spark/spark-3.5.5/spark-3.5.5-bin-hadoop3.tgz

In [30]:
!ls

drive  sample_data  spark-3.5.5-bin-hadoop3.tgz


In [31]:
!tar xf spark-3.5.5-bin-hadoop3.tgz

In [32]:
!pip install -q findspark

In [33]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.5-bin-hadoop3"

In [34]:
import findspark
findspark.init()

In [35]:
findspark.find()

'/content/spark-3.5.5-bin-hadoop3'

In [36]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Pyspark_SQL")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [37]:
spark

#IMPORTE DE LAS FUNCIONES NECESARIAS PARA REALIZAR LAS CONSULTAS

In [38]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, desc, row_number, avg

In [39]:
import pandas as pd

data_path = '/content/drive/MyDrive/BigData/Air_Traffic_Passenger_Statistics.csv'

data = spark.read.options(inferSchema='True',delimiter=',', header=True).csv(data_path)
data.take(3)

[Row(Activity Period=200507, Operating Airline='ATA Airlines', Operating Airline IATA Code='TZ', Published Airline='ATA Airlines', Published Airline IATA Code='TZ', GEO Summary='Domestic', GEO Region='US', Activity Type Code='Deplaned', Price Category Code='Low Fare', Terminal='Terminal 1', Boarding Area='B', Passenger Count=27271, Adjusted Activity Type Code='Deplaned', Adjusted Passenger Count=27271, Year=2005, Month='July'),
 Row(Activity Period=200507, Operating Airline='ATA Airlines', Operating Airline IATA Code='TZ', Published Airline='ATA Airlines', Published Airline IATA Code='TZ', GEO Summary='Domestic', GEO Region='US', Activity Type Code='Enplaned', Price Category Code='Low Fare', Terminal='Terminal 1', Boarding Area='B', Passenger Count=29131, Adjusted Activity Type Code='Enplaned', Adjusted Passenger Count=29131, Year=2005, Month='July'),
 Row(Activity Period=200507, Operating Airline='ATA Airlines', Operating Airline IATA Code='TZ', Published Airline='ATA Airlines', Publi

In [40]:
data

DataFrame[Activity Period: int, Operating Airline: string, Operating Airline IATA Code: string, Published Airline: string, Published Airline IATA Code: string, GEO Summary: string, GEO Region: string, Activity Type Code: string, Price Category Code: string, Terminal: string, Boarding Area: string, Passenger Count: int, Adjusted Activity Type Code: string, Adjusted Passenger Count: int, Year: int, Month: string]

#DESCRIPCIÓN DE LOS DATOS DEL FICHERO.

In [41]:
data.printSchema()

root
 |-- Activity Period: integer (nullable = true)
 |-- Operating Airline: string (nullable = true)
 |-- Operating Airline IATA Code: string (nullable = true)
 |-- Published Airline: string (nullable = true)
 |-- Published Airline IATA Code: string (nullable = true)
 |-- GEO Summary: string (nullable = true)
 |-- GEO Region: string (nullable = true)
 |-- Activity Type Code: string (nullable = true)
 |-- Price Category Code: string (nullable = true)
 |-- Terminal: string (nullable = true)
 |-- Boarding Area: string (nullable = true)
 |-- Passenger Count: integer (nullable = true)
 |-- Adjusted Activity Type Code: string (nullable = true)
 |-- Adjusted Passenger Count: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: string (nullable = true)



In [42]:
data.show(10)

+---------------+-----------------+---------------------------+-----------------+---------------------------+-------------+-------------------+------------------+-------------------+-------------+-------------+---------------+---------------------------+------------------------+----+-----+
|Activity Period|Operating Airline|Operating Airline IATA Code|Published Airline|Published Airline IATA Code|  GEO Summary|         GEO Region|Activity Type Code|Price Category Code|     Terminal|Boarding Area|Passenger Count|Adjusted Activity Type Code|Adjusted Passenger Count|Year|Month|
+---------------+-----------------+---------------------------+-----------------+---------------------------+-------------+-------------------+------------------+-------------------+-------------+-------------+---------------+---------------------------+------------------------+----+-----+
|         200507|     ATA Airlines|                         TZ|     ATA Airlines|                         TZ|     Domestic|    

# ¿Cuántas compañías diferentes aparecen en el fichero?

In [43]:
companydf = data.select('Operating Airline').distinct().count()
print("Número de compañías diferentes: ", companydf)

Número de compañías diferentes:  77


#¿Cuántos pasajeros tienen de media los vuelos de cada compañía?

In [44]:
avg_passengers_per_company = data.groupBy('Operating Airline').agg(avg('Passenger Count').alias("avg_passengers_count"))
avg_passengers_per_company.show()

+--------------------+--------------------+
|   Operating Airline|avg_passengers_count|
+--------------------+--------------------+
|          Icelandair|              2799.7|
|         Ameriflight|                 5.0|
|      Cathay Pacific|  17121.325581395347|
|          Aeromexico|   5463.822222222222|
|      Etihad Airways|   6476.088235294118|
| Philippine Airlines|  10248.635658914729|
|United Airlines -...|   48915.46750232126|
|    Turkish Airlines|   8162.416666666667|
| Swiss International|   6061.640287769784|
|    Independence Air|              6391.3|
|Miami Air Interna...|             107.375|
|          Air France|  11589.077519379845|
|      Japan Airlines|   6470.332046332046|
|    Midwest Airlines|              3883.0|
|      Atlas Air, Inc|                34.0|
|    JetBlue Airways |   35261.13963963964|
|       China Eastern|   5498.402777777777|
|   Mexicana Airlines|   7993.806451612903|
|         Air Canada |  18251.560109289618|
|       Allegiant Air|          

#Se eliminan los duplicados del fichero en el campo 'GEO region', manteniendo solamente los que tengan mayor número de pasajeros.

In [51]:
window_spec = Window.partitionBy("Geo region").orderBy(desc("Passenger Count"))
df_ranked = data.withColumn("rank", row_number().over(window_spec))
df_filtered = df_ranked.filter(col("rank") == 1).drop("rank")

#Unir la media de pasajeros al DataFrame original

In [52]:
df_final = df_filtered.join(avg_passengers_per_company, on="Operating Airline", how="left")

#Guardar el resultado en un nuevo CSV

In [55]:
df_final.write.option("header", "true").mode("overwrite").csv("resultado_final.csv")

df_final.show()

+--------------------+---------------+---------------------------+--------------------+---------------------------+-------------+-------------------+------------------+-------------------+-------------+-------------+---------------+---------------------------+------------------------+----+-------+--------------------+
|   Operating Airline|Activity Period|Operating Airline IATA Code|   Published Airline|Published Airline IATA Code|  GEO Summary|         GEO Region|Activity Type Code|Price Category Code|     Terminal|Boarding Area|Passenger Count|Adjusted Activity Type Code|Adjusted Passenger Count|Year|  Month|avg_passengers_count|
+--------------------+---------------+---------------------------+--------------------+---------------------------+-------------+-------------------+------------------+-------------------+-------------+-------------+---------------+---------------------------+------------------------+----+-------+--------------------+
|United Airlines -...|         200708|  